In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import math
import torch.utils.data as data_utils
from sklearn import metrics
import os
from sklearn.utils import shuffle
import torchvision

In [ ]:
#DATA

df = pd.read_json("/content/ID_data.json")

#df.head()



In [ ]:
#Using both q and dq as input features

T = np.array([i for i in df['effort'].to_numpy()])
q = np.array([i for i in df['position'].to_numpy()])
dq = np.array([i for i in df['velocity'].to_numpy()])

X = np.zeros((T.shape[0],6))
X[:,0:3] = q[:,4:7]
print(np.shape(X))
X[:,3:6] = dq[:,4:7]
Y = T[:,4:7]
#print(dq[:,4:7])
X = torch.from_numpy(X).float()
Y = torch.from_numpy(Y).float()

In [ ]:
#Using only q as input feature

T = np.array([i for i in df['effort'].to_numpy()])
q = np.array([i for i in df['position'].to_numpy()])
dq = np.array([i for i in df['velocity'].to_numpy()])

#print(dq)

X = np.zeros((T.shape[0],3))
X[:,0:3] = q[:,4:7]
Y = T[:,4:7]
X = torch.from_numpy(X).float()
Y = torch.from_numpy(Y).float()

In [ ]:
#Using only dq as input feature

T = np.array([i for i in df['effort'].to_numpy()])
q = np.array([i for i in df['position'].to_numpy()])
dq = np.array([i for i in df['velocity'].to_numpy()])

#print(dq)

X = np.zeros((T.shape[0],3))
X[:,0:3] = dq[:,4:7]
Y = T[:,4:7]
X = torch.from_numpy(X).float()
Y = torch.from_numpy(Y).float()

In [ ]:
#Normalization
loader = DataLoader(X, batch_size=len(X), num_workers=1)
data = next(iter(loader))
data.mean(), data.std()
normalised_X = (X-data.mean())/data.std()
print(normalised_X.mean(),normalised_X.std())


In [ ]:
#split data


tr = 0.6
vl = 0.25
ts = 0.15

n = T.shape[0]
tr_n = round(tr*n)
vl_n = round((n-tr_n)*(vl/(ts+vl)))
ts_n = n-(tr_n+vl_n)

X = normalised_X

X_training = X[0:tr_n,:]
Y_training = Y[0:tr_n,:] 

X_val =   X[tr_n:tr_n+vl_n,:]  
Y_val =   Y[tr_n:tr_n+vl_n,:]  

X_test =  X[tr_n+vl_n:-1,:] 
Y_test =  Y[tr_n+vl_n:-1,:] 


In [ ]:
#Using both dq and q as input feature
#Defining the nn model
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(6, 64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,3)
)


#print(X_training.shape,Y_training.shape)

#Define loss function
loss_fn = nn.MSELoss()

#Defining the optimiser
opt = optim.Adam(net.parameters(), lr=1e-5)
#opt = optim.SGD(net.parameters(), lr=1e-3)

dataset_train = data_utils.TensorDataset(X_training,Y_training)
dataset_val = data_utils.TensorDataset(X_val,Y_val)

N_samples = len(dataset_train)
EpochMax = 1000
batch_size = 32
iter = math.ceil(N_samples/batch_size)

#Seprating data to batches
train_loader = DataLoader(dataset= dataset_train, batch_size=batch_size, shuffle=True,num_workers=1)
val_loader = DataLoader(dataset= dataset_val, batch_size=1, shuffle=True,num_workers=1)

loss = 0.0
best_valid_loss = 1000.0
PATH = "state_dict_net.pt"

for epoch in range(EpochMax):
  loss_batch = 0.0
  print(f'Starting epoch {epoch+1}')
  for b,(inputs, outputs) in enumerate(train_loader,0):
    X_batch, Y_batch = (inputs, outputs)
    
    #Clear gradiants
    opt.zero_grad()
    
    #Forward pass
    y_hat_batch = net(X_batch)
    
    loss_batch = loss_fn(y_hat_batch, Y_batch)
    #print(loss_batch)
    #Calculate gradians
    loss_batch.backward()

    #Update Weights
    opt.step()
   
    loss += loss_batch
  training_loss   = loss/iter
  
  #Validation Loop
  loss_valid = 0.0
  with torch.no_grad():
    for X_val, Y_val in val_loader:
      y_hat_val = net(X_val)
      loss_valid = loss_fn(y_hat_val, Y_val)
      if loss_valid<best_valid_loss:
       best_valid_loss = loss_valid
       torch.save(net.state_dict(),PATH)
  loss = 0

In [ ]:
#Using only q or only dq as input feature
#Defining the nn model
net = nn.Sequential(
    nn.Linear(3, 64),
    nn.Tanh(),
    nn.Linear(64,64),
    nn.Tanh(),
    nn.Linear(64,3)
)


#print(X_training.shape,Y_training.shape)

#Define loss function
loss_fn = nn.MSELoss()

#Defining the optimiser
opt = optim.Adam(net.parameters(), lr=1e-5)
#opt = optim.SGD(net.parameters(), lr=1e-3)

dataset = data_utils.TensorDataset(X_training,Y_training)

N_samples = len(dataset)
EpochMax = 1000
batch_size = 32
iter = math.ceil(N_samples/batch_size)

#Seprating data to batches
train_loader = DataLoader(dataset= dataset, batch_size=batch_size, shuffle=True)

loss = 0.0
best_valid_loss = 1000.0
PATH = "state_dict_net.pt"

for epoch in range(EpochMax):
  loss_batch = 0.0
  for b,(inputs, outputs) in enumerate(train_loader,0):
    X_batch, Y_batch = (inputs, outputs)
    
    #Clear gradiants
    opt.zero_grad()
    
    #Forward pass
    y_hat_batch = net(X_batch)
    
    loss_batch = loss_fn(y_hat_batch, Y_batch)
    print(loss_batch)
    #Calculate gradians
    loss_batch.backward()

    #Update Ws
    opt.step()
   
    loss += loss_batch
  training_loss   = loss/iter
  
  #Validation Loop
  loss_valid = 0.0
  with torch.no_grad():
    y_hat_val = net(X_val)
    loss_valid = loss_fn(y_hat_val, Y_val)
    if loss_valid<best_valid_loss:
      best_valid_loss = loss_valid
      torch.save(net.state_dict(),PATH)
  loss = 0


In [ ]:
def mse(actual, predicted):
    return metrics.mean_squared_error(actual, predicted)

def rmse(actual, predicted):
    return np.sqrt(mse(actual, predicted))


def nrmse(actual, predicted):
    return rmse(actual, predicted) / (actual.max() - actual.min())

In [ ]:
net.load_state_dict(torch.load(PATH))
y_hat_test = torch.tensor(net(X_test))

#print(Y_test.shape,y_hat_test)
y_hat_test.numpy()
Y_test.numpy()

print('test rmse',rmse(Y_test,y_hat_test))
print('test nrmse',nrmse(Y_test,y_hat_test))

y_hat_tr = torch.tensor(net(X_training))
y_hat_tr.numpy()
Y_test.numpy()

print('training rmse',rmse(Y_training,y_hat_tr))
print('nrmse',nrmse(Y_training,y_hat_tr))